In [1]:
!pip install Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
from sklearn.metrics import jaccard_score
import Levenshtein

In [4]:
#download dataset from : https://www.kaggle.com/datasets/rmisra/news-category-dataset

# Read the JSON file line by line
lines = []
with open('/content/drive/MyDrive/datasets/News_Category_Dataset_v3.json', 'r') as file:
    for line in file:
        line = line.strip()
        if line:
            lines.append(line)

# Concatenate the lines into a single string
data = "[" + ",".join(lines) + "]"

# Read the concatenated JSON data into a DataFrame
df = pd.read_json(data, orient='columns')

# Display the DataFrame
df


,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22
...,...,...,...,...,...,...
209522,https://www.huffingtonpost.com/entry/rim-ceo-t...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...,"Reuters, Reuters",2012-01-28
209523,https://www.huffingtonpost.com/entry/maria-sha...,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",,2012-01-28
209524,https://www.huffingtonpost.com/entry/super-bow...,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...",,2012-01-28
209525,https://www.huffingtonpost.com/entry/aldon-smi...,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,,2012-01-28


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209527 entries, 0 to 209526
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   link               209527 non-null  object        
 1   headline           209527 non-null  object        
 2   category           209527 non-null  object        
 3   short_description  209527 non-null  object        
 4   authors            209527 non-null  object        
 5   date               209527 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 9.6+ MB


In [6]:
df['date'] = pd.to_datetime(df['date'])

In [7]:
df.isnull().sum()

link                 0
headline             0
category             0
short_description    0
authors              0
date                 0
dtype: int64

In [8]:
df[df.duplicated()]

,link,headline,category,short_description,authors,date
67677,https://www.huffingtonpost.comhttp://www.mothe...,"On Facebook, Trump's Longtime Butler Calls For...",POLITICS,"Anthony Senecal, who worked as Donald Trump's ...",,2016-05-12
67923,https://www.huffingtonpost.comhttp://gizmodo.c...,Former Facebook Workers: We Routinely Suppress...,TECH,Facebook workers routinely suppressed news sto...,,2016-05-09
70239,https://www.huffingtonpost.comhttp://www.cnbc....,"On Equal Pay Day, The Gap Is Still Too Wide",WOMEN,Equal Pay Day falls on April 12 in 2016. It's ...,,2016-04-12
139830,https://www.huffingtonpost.comhttp://www.cnn.c...,The World's Most Dangerous Workout?,WELLNESS,"Is the ""sport of fitness"" the world's most dan...",,2014-02-10
144409,https://www.huffingtonpost.comhttp://www.upwor...,Some People Call It 'The Best Anti-Smoking Ad ...,WELLNESS,Almost all smokers know cigarettes are bad for...,,2013-12-22
145142,https://www.huffingtonpost.comhttp://www.weath...,10 Cities That Could Run Out Of Water - Weathe...,ENVIRONMENT,"Securing access to plentiful, renewable source...",,2013-12-15
178155,https://www.huffingtonpost.comhttp://www.busin...,Google Is Attacking Apple From The Inside Out ...,TECH,After years of hammering away at Apple's share...,,2013-01-01
187329,https://www.huffingtonpost.comhttp://www.nytim...,"Eating For Health, Not Weight",WELLNESS,Almost half of Americans are on a diet -- not ...,,2012-09-23
194596,https://www.huffingtonpost.comhttp://blogs.wsj...,Apple Removes Green EPEAT Electronics Certific...,TECH,Apple has pulled its products off the U.S. gov...,,2012-07-07
194598,https://www.huffingtonpost.comhttp://www.theda...,Microsoft's $6.2 Billion Writedown Shows It's ...,TECH,Fighting for online advertising dominance with...,,2012-07-07


In [9]:
df.drop_duplicates(inplace=True)

In [10]:
# Select relevant columns for analysis
data = df[['category', 'headline', 'short_description']]

In [11]:
# Preprocess the data
data['text'] = data['headline'] + ' ' + data['short_description']

<ipython-input-11-0c1b2e49763f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['headline'] + ' ' + data['short_description']


In [12]:
# Vectorize the text data
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['text'])

In [ ]:
# Function to calculate similarity scores using cosine similarity and Levenshtein distance
def calculate_similarity(query):
    # Vectorize the query
    query_vector = vectorizer.transform([query])

    # Calculate cosine similarity scores
    cosine_sim = cosine_similarity(X, query_vector).flatten()

    # Calculate Levenshtein distance scores
    levenshtein_sim = []
    for text in data['text']:
        levenshtein_sim.append(distance(query, text))

    # Combine the similarity scores
    similarity_scores = (cosine_sim + levenshtein_sim) / 2

    return similarity_scores

In [ ]:
# Function to find the most similar data to the query
def find_similar_data(query, top_n=5):
    # Calculate similarity scores
    similarity_scores = calculate_similarity(query)

    # Find the indices of the top similar data points
    top_indices = similarity_scores.argsort()[-top_n:][::-1]

    # Return the top similar data points
    similar_data = data.iloc[top_indices]

    return similar_data




In [22]:
# Example usage
query = "Global warming maybe more dangerous than we think says the new research "

similar_data = find_similar_data(query)
similar_data

,category,headline,short_description,text
66816,POLITICS,Sunday Roundup,This week the nation watched as the #NeverTrum...,Sunday Roundup This week the nation watched as...
109802,WORLDPOST,Weekend Roundup: Laughing at God,The first principle of an open society is not ...,Weekend Roundup: Laughing at God The first pri...
61264,POLITICS,Sunday Roundup,This week the nation came one step closer to g...,Sunday Roundup This week the nation came one s...
109238,WORLDPOST,"Weekend Roundup: One Year On, The WorldPost Ha...",The WorldPost was launched one year ago in Dav...,"Weekend Roundup: One Year On, The WorldPost Ha..."
63109,POLITICS,Sunday Roundup,"This week, the nation was reminded, in ways bo...","Sunday Roundup This week, the nation was remin..."
